# Data loading and observing
Load the **life_expectancy.csv** dataset into a pandas DataFrame by first importing pandas, and then using the `pandas.read_csv()` function to load the file and assign the resulting DataFrame to a variable called `dataset`.

In [2]:
import pandas as pd

dataset = pd.read_csv("life_expectancy.csv")

Observe the data by printing the first five entries in the DataFrame `dataset` by using the `DataFrame.head()` function. Make sure to see what the columns are and what the data types are. Locate the feature we would like to predict: `life expectancy`. Use 7DataFrame.describe()7 to see the summary statistics of the data.

In [6]:
dataset.head()

,Country,Year,Status,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling,Life expectancy
0,Afghanistan,2015,Developing,263.0,62,0.01,71.279624,65.0,1154,19.1,83,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1,65.0
1,Afghanistan,2014,Developing,271.0,64,0.01,73.523582,62.0,492,18.6,86,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0,59.9
2,Afghanistan,2013,Developing,268.0,66,0.01,73.219243,64.0,430,18.1,89,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9,59.9
3,Afghanistan,2012,Developing,272.0,69,0.01,78.184215,67.0,2787,17.6,93,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8,59.5
4,Afghanistan,2011,Developing,275.0,71,0.01,7.097109,68.0,3013,17.2,97,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5,59.2


In [7]:
dataset.columns

Index(['Country', 'Year', 'Status', 'Adult Mortality', 'infant deaths',
       'Alcohol', 'percentage expenditure', 'Hepatitis B', 'Measles ', ' BMI ',
       'under-five deaths ', 'Polio', 'Total expenditure', 'Diphtheria ',
       ' HIV/AIDS', 'GDP', 'Population', ' thinness  1-19 years',
       ' thinness 5-9 years', 'Income composition of resources', 'Schooling',
       'Life expectancy'],
      dtype='object')

In [5]:
dataset.describe()

,Year,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling,Life expectancy
count,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000,2.938000e+03,2938.000000,2938.000000,2938.000000,2938.000000,2938.000000
mean,2007.518720,164.725664,30.303948,4.546875,738.251295,83.022124,2419.592240,38.381178,42.035739,82.617767,5.924098,82.393125,1.742103,6611.523863,1.023085e+07,4.821886,4.852144,0.630362,12.009837,69.234717
std,4.613841,124.086215,117.926501,3.921946,1987.914858,22.996984,11467.272489,19.935375,160.445548,23.367166,2.400770,23.655562,5.077785,13296.603449,5.402242e+07,4.397621,4.485854,0.205140,3.265139,9.509115
min,2000.000000,1.000000,0.000000,0.010000,0.000000,1.000000,0.000000,1.000000,0.000000,3.000000,0.370000,2.000000,0.100000,1.681350,3.400000e+01,0.100000,0.100000,0.000000,0.000000,36.300000
25%,2004.000000,74.000000,0.000000,1.092500,4.685343,82.000000,0.000000,19.400000,0.000000,78.000000,4.370000,78.000000,0.100000,580.486996,4.189172e+05,1.600000,1.600000,0.504250,10.300000,63.200000
50%,2008.000000,144.000000,3.000000,3.755000,64.912906,92.000000,17.000000,43.500000,4.000000,93.000000,5.755000,93.000000,0.100000,1766.947595,1.386542e+06,3.300000,3.300000,0.677000,12.300000,72.100000
75%,2012.000000,227.000000,22.000000,7.390000,441.534144,96.000000,360.250000,56.100000,28.000000,97.000000,7.330000,97.000000,0.800000,4779.405190,4.584371e+06,7.100000,7.200000,0.772000,14.100000,75.600000
max,2015.000000,723.000000,1800.000000,17.870000,19479.911610,99.000000,212183.000000,87.300000,2500.000000,99.000000,17.600000,99.000000,50.600000,119172.741800,1.293859e+09,27.700000,28.600000,0.948000,20.700000,89.000000


Drop the Country column from the DataFrame using the `DataFrame` drop method. Why? To create a predictive model, knowing from which country data comes can be confusing and it is not a column we can generalize over. We want to learn a general pattern for all the countries, and not only those dependent on specific countries.

In [14]:
dataset = dataset.drop(columns = ["Country"])

In the next two steps, you will split the data into labels and features. Labels are contained in the “Life expectancy” column. It’s the final column in the DataFrame. Create a new variable called `labels`. Use `iloc` indexing to assign the final column of `dataset` to it.

In [15]:
labels = dataset.iloc[:, -1]
labels

0       65.0
1       59.9
2       59.9
3       59.5
4       59.2
        ... 
2933    44.3
2934    44.5
2935    44.8
2936    45.3
2937    46.0
Name: Life expectancy, Length: 2938, dtype: float64

Features span from the first column to the last column (not including it, since it’s a label column in our dataset). Create a new variable called `features`. Use `iloc` indexing to assign a subset of columns from first to last (not including the last column) to it.

In [16]:
features = dataset.iloc[:, : -1]
features

,Year,Status,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,2015,Developing,263.0,62,0.01,71.279624,65.0,1154,19.1,83,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,2014,Developing,271.0,64,0.01,73.523582,62.0,492,18.6,86,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,2013,Developing,268.0,66,0.01,73.219243,64.0,430,18.1,89,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,2012,Developing,272.0,69,0.01,78.184215,67.0,2787,17.6,93,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,2011,Developing,275.0,71,0.01,7.097109,68.0,3013,17.2,97,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933,2004,Developing,723.0,27,4.36,0.000000,68.0,31,27.1,42,67.0,7.13,65.0,33.6,454.366654,12777511.0,9.4,9.4,0.407,9.2
2934,2003,Developing,715.0,26,4.06,0.000000,7.0,998,26.7,41,7.0,6.52,68.0,36.7,453.351155,12633897.0,9.8,9.9,0.418,9.5
2935,2002,Developing,73.0,25,4.43,0.000000,73.0,304,26.3,40,73.0,6.53,71.0,39.8,57.348340,125525.0,1.2,1.3,0.427,10.0
2936,2001,Developing,686.0,25,1.72,0.000000,76.0,529,25.9,39,76.0,6.16,75.0,42.1,548.587312,12366165.0,1.6,1.7,0.427,9.8


# Data Preprocessing
When you observed your dataset you probably noticed that some columns are categorical. We learned in this lesson that categorical columns need to be converted into numerical columns using methods such as one-hot-encoding. Use `pandas.get_dummies(DataFrame)` to apply one-hot-encoding on all the categorical columns. Assign the result of the encoding back to the `features` variable.

In [19]:
features = pd.get_dummies(features)
features

,Year,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling,Status_Developed,Status_Developing
0,2015,263.0,62,0.01,71.279624,65.0,1154,19.1,83,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1,0,1
1,2014,271.0,64,0.01,73.523582,62.0,492,18.6,86,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0,0,1
2,2013,268.0,66,0.01,73.219243,64.0,430,18.1,89,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9,0,1
3,2012,272.0,69,0.01,78.184215,67.0,2787,17.6,93,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8,0,1
4,2011,275.0,71,0.01,7.097109,68.0,3013,17.2,97,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933,2004,723.0,27,4.36,0.000000,68.0,31,27.1,42,67.0,7.13,65.0,33.6,454.366654,12777511.0,9.4,9.4,0.407,9.2,0,1
2934,2003,715.0,26,4.06,0.000000,7.0,998,26.7,41,7.0,6.52,68.0,36.7,453.351155,12633897.0,9.8,9.9,0.418,9.5,0,1
2935,2002,73.0,25,4.43,0.000000,73.0,304,26.3,40,73.0,6.53,71.0,39.8,57.348340,125525.0,1.2,1.3,0.427,10.0,0,1
2936,2001,686.0,25,1.72,0.000000,76.0,529,25.9,39,76.0,6.16,75.0,42.1,548.587312,12366165.0,1.6,1.7,0.427,9.8,0,1


Split your data into training set and test sets using the `sklearn.model_selection.train_test_split()` function. Assign

* the training features to a variable called `features_train`
* training labels to a variable called `labels_train`
* test data to a variable called `features_test`
* test labels to a variable called `labels_test`.

You can choose any percentage for `test_size` and any value for `random_state`.

In [21]:
from sklearn.model_selection import train_test_split

features_train, features_test, labels_train, labels_test = train_test_split(
    features, 
    labels, 
    test_size = 0.33,         # we chose the test size to be 33% of the total data,
    random_state = 42
)

The next step is to standardize/normalize your numerical features. You can pick whichever method you want. In this step, create a `sklearn.compose.ColumnTransformer` instance called `ct` to set up the normalization/standardization procedure. When initializing `ColumnTransformer` make sure to list all of the numerical features you have in your dataset. Or use `DataFrame.select_dtypes()` to select `float64` or `int64` feature types automatically.

In [22]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.compose import ColumnTransformer

numerical_features = features.select_dtypes(
    include = ['float64', 'int64']
)
numerical_columns = numerical_features.columns

ct = ColumnTransformer(
    [(
        "only numeric", 
        StandardScaler(), 
        numerical_columns 
    )], 
    remainder = "passthrough"
)

Fit your instance `ct` of `ColumnTransformer` to the training data and at the same time transform it by using the `ColumnTransformer.fit_transform()` method. Assign the result to a variable called `features_train_scaled`.

In [23]:
features_train_scaled = ct.fit_transform(features_train) 

Transform your test data instance `features_test` using the trained `ColumnTransformer` instance `ct`. Assign the result to a variable called `features_test_scaled`.

In [24]:
features_test_scaled = ct.transform(features_test)  

# Building the model
Create an instance of the `tensorflow.keras.models.Sequential` model called `my_model`.

In [27]:
from tensorflow.keras.models import Sequential

my_model = Sequential()

Create the input layer to the network model using `tf.keras.layers.InputLayer` with the shape corresponding to the number of features in your dataset. Assign the resulting input layer to a variable called `input`.

In [28]:
from tensorflow.keras.layers import InputLayer

input = InputLayer(input_shape = (features.shape[1], ))

Add the input layer from the previous step to the model instance `my_model`.

In [29]:
my_model.add(input)

Add one `keras.layers.Dense` hidden layer with any number of hidden units you wish. Use the `relu` activation function.

In [30]:
from tensorflow.keras.layers import Dense

my_model.add(
    Dense(
        64, 
        activation = "relu" 
    )
)

Add a `keras.layers.Dense` output layer with one neuron since you need a single output for a regression prediction.

In [31]:
my_model.add(Dense(1))


Print the summary of the model using the `Sequential.summary()` method.

In [32]:
print(my_model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                1408      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 1,473
Trainable params: 1,473
Non-trainable params: 0
_________________________________________________________________
None


# Initializing the optimizer and compiling the model
Create an instance of the `Adam` optimizer with the learning rate equal to 0.01.

In [33]:
from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate = 0.01)

Once your optimizer is initialized, compile the model using the `Sequential.compile()` method.

Assign the following parameters:

* For `loss` use the Mean Squared Error (`mse`)
* For `metrics` use the Mean Absolute Error (`mae`)
* For `opt` (the optimizer parameters) use the instance of the optimizer you created in the previous step.

In [34]:
my_model.compile(
    loss = "mse",  
    metrics = ["mae"], 
    optimizer = opt
)

# Fit and evaluate the model
Train your model with the `Sequential.fit()` method by passing it the following parameters:

* your preprocessed training data
* training labels
* number of epochs equal to 40
* batch size equal to 1
* verbose set to 1

In [35]:
my_model.fit(
    features_train_scaled,
    labels_train,
    epochs = 40,
    batch_size = 1,
    verbose = 1
)

Epoch 1/40
1968/1968 [==============================] - 3s 2ms/step - loss: 167.7844 - mae: 6.8683
Epoch 2/40
1968/1968 [==============================] - 3s 2ms/step - loss: 30.7954 - mae: 4.0207
Epoch 3/40
1968/1968 [==============================] - 3s 2ms/step - loss: 23.5166 - mae: 3.6055
Epoch 4/40
1968/1968 [==============================] - 3s 2ms/step - loss: 22.9636 - mae: 3.3925
Epoch 5/40
1968/1968 [==============================] - 3s 2ms/step - loss: 22.3851 - mae: 3.2759
Epoch 6/40
1968/1968 [==============================] - 3s 2ms/step - loss: 18.6724 - mae: 3.1143
Epoch 7/40
1968/1968 [==============================] - 3s 2ms/step - loss: 18.0762 - mae: 3.0505
Epoch 8/40
1968/1968 [==============================] - 3s 2ms/step - loss: 19.9622 - mae: 3.0616
Epoch 9/40
1968/1968 [==============================] - 3s 2ms/step - loss: 15.2619 - mae: 2.7682
Epoch 10/40
1968/1968 [==============================] - 3s 2ms/step - loss: 14.0126 - mae: 2.7418
Epoch 11/40
1968/1

Using the `Sequential.evaluate()` method, evaluate your trained model on the preprocessed test data set, and with the test labels. Set verbose to 0. Store the result of the evaluation in two variables: `res_mse` and `res_mae`.

In [37]:
res_mse, res_mae = my_model.evaluate(
  features_test_scaled, 
  labels_test, 
  verbose = 0
)

Print your final loss (RMSE) and final metric (MAE) to check the predictive performance on the test set.

In [38]:
print("RMS: ", res_mse)
print("MAE: ", res_mae)

RMS:  7.8520588874816895
MAE:  2.0885465145111084
